# Data Analysis

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset

id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detected 0 bad samples
Detected 0 bad samples


In [9]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    try:
        events = get_readable_events(input_ids)
    except Exception as e:
        print(f"Error: {e}")
        print(f"Input IDs: {input_ids}")
        continue
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 50)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df.to_csv("events.csv", index=False)
df

dataset id_train                                   id_test              ...  \
key         misc onset   dur inst pitch  antic vel    misc onset   dur  ...   
0             AR                                        AR              ...   
1                  0.0  0.09    0   A#4  False               0.0  0.05  ...   
2                 4.84  0.35    0    G2   True               0.0  0.06  ...   
3                 4.85  0.23    0    G1   True              0.11  0.06  ...   
4                 4.88  0.29    0    G3   True              0.25  0.05  ...   
5                 0.01  0.11    0    D4  False              0.37  0.08  ...   
6                  0.4  0.12    0    C6  False              0.47  0.04  ...   
7                 0.41  0.18    0    C5  False              0.47  0.05  ...   
8                 0.85  0.13    0   D#5  False              0.49  0.07  ...   
9                 1.28  0.15    0    F5  False              0.64  0.04  ...   
10                5.91  0.16    0   A#5   True              0.73  0.07  ...   
11                5.92  0.12    0    G5   True              0.83  0.06  ...   
12                5.94  0.23    0    G4   True              0.95  0.04  ...   
13                5.95  0.29    0   A#3   True              0.96  0.07  ...   
14                5.95  0.64    0    G3   True              0.96  0.05  ...   
15                5.95  0.21    0    D4   True              0.97  0.04  ...   
16                1.78  0.21    0   G#4  False              1.07  0.05  ...   
17                 6.3  0.19    0   A#3   True              1.19   0.1  ...   
18                6.31   0.2    0    G5   True              1.31  0.06  ...   
19                6.31  0.11    0    D5   True              1.44  0.07  ...   
20                6.32   0.2    0    G4   True              1.46  0.04  ...   
21                6.32  0.16    0    D6   True              1.46  0.06  ...   
22                6.32  0.31    0   A#5   True              1.55  0.06  ...   
23                 6.7  0.24    0    E4   True              1.66  0.08  ...   
24                 6.7  0.13    0    C6   True              1.77  0.05  ...   
25                6.72  0.17    0    E5   True               1.9  0.07  ...   
26                6.72  0.18    0    C4   True              1.91  0.07  ...   
27                1.78  0.24    0    F4  False              1.91  0.05  ...   
28                2.44  0.76    0    G3  False              2.04  0.04  ...   
29                7.09   0.1    0    E5   True              2.06   0.0  ...   
30                 7.1   0.1    0    E4   True              2.19  0.03  ...   
31                7.11  0.08    0    C6   True              2.27  0.07  ...   
32                7.11  0.15    0    C5   True              2.39  0.04  ...   
33                7.13  0.06    0    G3   True               2.4  0.08  ...   
34                3.26  0.26    0    G5  False              2.52  0.09  ...   
35                7.56  0.12    0    C5   True              2.61  0.09  ...   
36                7.56  0.17    0   G#5   True              2.74  0.07  ...   
37                7.56  0.48    0   G#3   True              2.87  0.06  ...   
38                7.56  0.12    0   G#4   True              2.88  0.05  ...   
39                7.57  0.17    0    C4   True               2.9  0.03  ...   
40                 8.0  0.11    0    C6   True              3.01  0.07  ...   
41                8.01  0.16    0   G#5   True              3.13  0.09  ...   
42                8.02  0.15    0    C5   True              3.23   0.1  ...   
43                8.02  0.19    0   D#5   True              3.37  0.05  ...   
44                8.03  0.24    0   D#4   True              3.37  0.12  ...   
45          REST  4.26   0.0             False              3.38  0.06  ...   
46                8.49  0.16    0   A#5   True              3.39  0.06  ...   
47                 8.5  0.22    0   C#5   True              3.48   0.1  ...   
48                 8.5  0.24    0   A#4   True    

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value <0.001
{'skewness': np.float64(172381.575), 'kurtosis': np.float64(-2.049), 'skewness_p': 0.122, 'kurtosis_p': 0.04, 'royston_p': 0.157, 'hz_p_value': 0.001}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

{0: array([[-1.5858957e-04, -2.2780974e-03, -5.4641580e-03, ...,
          3.6041990e-02,  3.7796646e-02,  3.8871054e-02],
        [ 7.5466487e-05,  6.6216319e-04, -6.6342456e-03, ...,
          3.2185879e-02,  3.5957586e-02,  3.7327852e-02],
        [-1.6285133e-03, -3.2915168e-03, -2.0713529e-03, ...,
          3.7087522e-02,  3.7383165e-02,  3.8296748e-02],
        ...,
        [-1.7962824e-03,  2.3272866e-03, -6.3806227e-03, ...,
          3.4933787e-02,  3.7223965e-02,  3.6779165e-02],
        [ 1.0697738e-03, -6.2378547e-03,  3.5575293e-03, ...,
          3.6529809e-02,  3.8544346e-02,  3.8415130e-02],
        [ 6.0749738e-03, -7.5836191e-03,  1.7723180e-03, ...,
          3.4965035e-02,  3.5934646e-02,  3.8681839e-02]],
       shape=(4550, 2048), dtype=float32),
 1: array([[-0.0273591 ,  0.01079079, -0.05596501, ...,  0.09060913,
          0.09643698,  0.12677993],
        [-0.05586493,  0.00803197, -0.06769733, ...,  0.08572365,
          0.08836669,  0.12163065],
        [-0.0

In [6]:
import numpy as np
import pandas as pd
from constants.file_format import get_extract_layers_file_path
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = get_extract_layers_file_path(
        dataset_name="id_train_dataset",
        pooling_function_name=pooling_function.__name__,
        layer_idx=layer_idx,
    )
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing
p_value <0.001


Processing layers:   4%|▍         | 1/25 [00:03<01:14,  3.09s/it]

Processing layer 1
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:06<01:09,  3.02s/it]

...

p_value <0.001
Processing layer 23
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing
p_value <0.001


Processing layers:  96%|█████████▌| 24/25 [01:13<00:03,  3.05s/it]

Processing layer 24
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers: 100%|██████████| 25/25 [01:16<00:00,  3.04s/it]

p_value <0.001


In [7]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,909739.145,216.306,0.001,0.001,0.001,0.001
1,991356.101,218.161,0.001,0.001,0.001,0.001
2,1070157.759,163.435,0.001,0.001,0.001,0.001
3,1129641.367,159.503,0.001,0.001,0.001,0.001
4,1259987.950,185.162,0.001,0.001,0.001,0.001
5,1315038.299,206.776,0.001,0.001,0.001,0.001
6,1177959.743,180.221,0.001,0.001,0.001,0.001
7,1158573.736,173.303,0.001,0.001,0.001,0.001
8,1086299.071,142.337,0.001,0.001,0.001,0.001
9,986077.163,110.269,0.001,0.001,0.001,0.001


In [8]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)